In [ ]:
import pandas as pd
from dns.resolver import resolve
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map, process_map
tqdm.pandas()
# From https://host.io/rankings
df = pd.read_csv("host10m.csv.gz", names=["domain", "rank"]).head(10000)
df

In [ ]:
bad_IPs = "65.9.141.129,65.9.141.14,65.9.141.29,65.9.141.47,65.9.141.57,65.9.141.82,65.9.141.86,65.9.141.92".split(",")
bad_IPs

In [ ]:
def safe_resolve(domain):
  try:
    return resolve(domain, lifetime=30)
  except Exception as e:
    print(e)
#df["response"] = process_map(safe_resolve, df.domain, chunksize=10)
df["response"] = df.domain.progress_apply(safe_resolve)

In [ ]:
def calc_failure_rate(response):
    return sum([str(r) in bad_IPs for r in response])/len(response)

df["failure_rate"] = df.response.head(100).apply(calc_failure_rate)
df[df.failure_rate > 0].sort_values("failure_rate")